# Model 2

This notebboks shows a basic usage example of the `Model2` for the railway scheduling problem. This model solves the railwayscheduling problem using the branch & bounds algorithm implemented in Gurobi with the addition of the initial solution generation through the simulated annealing algorith, the usage of additional valid inequalities to tighted the feasible space and the inclusion of the following cutting plane generators:

- Boolean quadratic polytope cuts
- Clique cuts
- Cover cuts
- Flow cover cuts
- Flow path cuts
- Gomori cuts
- GUB cover cuts
- Implied bound cuts
- Lift-and-project cuts
- MIR cuts
- Mod-k cuts
- Network cuts
- Relax-and-lift cuts
- Strong-CG cuts
- $\{0, \frac{1}{2}\}$ cuts

To initialize the model the following imports are needed.

In [1]:
import os
import sys
import numpy as np
from railway import *

We then change the directory to the root folder of the project in order to correctly load the datasets.\
⚠ **WARNING**: change the path below, which might differ from mine, according to your file system setting in order to correctly load the datasets.

In [2]:
# Root folder directory
ROOT_DIR = "/home/marco/railway-scheduling"

# Set the current directory to root directory
os.chdir(ROOT_DIR)
sys.path.append(os.getcwd())
print(f"Current working directory: 📂 {os.getcwd()}")

Current working directory: 📂 /home/marco/railway-scheduling


We can then initialize the railway scheduling problem by loading one of the datasets present in the `datasets/` folder. To do so we need to define the parameters:

- `N`: number of nodes (stations), with $N \in \{10, 20, 40\}$
- `T`: time horizon, with $T \in \{10, 50, 100\}$
- `J`: number of jobs, with $J \in \{10, 40, 80\}$
- `P`: total number of passengers per arc, currently only $P = 2000$ is available
- `K`: number of alternative routes, currently only $K = 3$ is available

And the problem can be instantiated as follows. Notice that we use the same class as in the `Model0` notebook, the difference will be in which methods we call before the optimization step.

In [3]:
# Define problem parameters
N = 10
T = 10
J = 80
P = 2000
K = 3

# Name of the file to load
FILENAME = f"datasets/railway_N{N}_T{T}_J{J}_P{P}_K{K}.json"

# Instantiate the Railway object
rail = Railway.load(FILENAME)
rail.model.setParam('TimeLimit', 60)

# Print the model in summary
print(rail)

Set parameter Username
Set parameter LicenseID to value 2585388
Academic license - for non-commercial use only - expires 2025-11-15


Set parameter TimeLimit to value 60
Railway scheduling problem

Parameters:
N:  10 stations
T:  10 periods
J:  80 jobs
P:  2000 passengers
K:  3 alternative routes
Aj: 80 jobs with arcs
Ja: 37 arcs with jobs
C:  0 arcs unavailable simultaneously

Optimization model:
Variables:   0
Constraints: 0
Objective:   0.0
Status:      LOADED



Alternatively the model can be initialized using the default constructor rather than importing it from a dataset file:

```python
rail = Railway(N, T, J, P, K)
```

But then the other problem parameters such as stations coordinates or passengers' demands must be randomly generated with the `generate()` method. See the `apps/generate.py` file for an example on how to do that.

An initial solution can then be generated and set using the simulated annealing meta-heuristic as follows.

In [4]:
# Generate initial solution through simulated annealing
_ = rail.simulated_annealing()

# Set the initial solution
rail.set_solution()

# Print initial solution
print(f"Initial solution with SA:\t{rail.S}")

Initial solution with SA:	{1: 6, 2: 1, 3: 6, 4: 4, 5: 1, 6: 4, 7: 4, 8: 1, 9: 5, 10: 9, 11: 5, 12: 5, 13: 3, 14: 7, 15: 4, 16: 3, 17: 6, 18: 4, 19: 3, 20: 6, 21: 6, 22: 6, 23: 4, 24: 7, 25: 3, 26: 7, 27: 3, 28: 4, 29: 6, 30: 2, 31: 1, 32: 1, 33: 9, 34: 8, 35: 8, 36: 6, 37: 3, 38: 2, 39: 1, 40: 5, 41: 7, 42: 3, 43: 4, 44: 5, 45: 8, 46: 1, 47: 2, 48: 8, 49: 9, 50: 2, 51: 7, 52: 9, 53: 9, 54: 7, 55: 8, 56: 8, 57: 5, 58: 2, 59: 7, 60: 2, 61: 6, 62: 7, 63: 6, 64: 9, 65: 7, 66: 6, 67: 8, 68: 4, 69: 9, 70: 3, 71: 4, 72: 4, 73: 7, 74: 5, 75: 3, 76: 2, 77: 1, 78: 7, 79: 8, 80: 5}


Then, we can set the problem constraints and the objective function to minimize with the dedicated methods.

In [5]:
# Set constraints
rail.set_constraints()

# Set objective
rail.set_objective()

For this model we then set the valid inequalities to be used in the optimization processand the cutting planes generators with the dedicated implemented methods.

In [6]:
# Set valid inequalities
rail.set_valid_inequalities()

# Set cuttings planes
rail.set_cutting_planes()

Set parameter Presolve to value 1
Set parameter Cuts to value 0
Set parameter BQPCuts to value -1
Set parameter CliqueCuts to value -1
Set parameter CoverCuts to value -1
Set parameter FlowCoverCuts to value -1
Set parameter FlowPathCuts to value -1
Set parameter GomoryPasses to value -1
Set parameter GUBCoverCuts to value -1
Set parameter ImpliedCuts to value -1
Set parameter LiftProjectCuts to value -1
Set parameter MIRCuts to value -1
Set parameter ModKCuts to value -1
Set parameter NetworkCuts to value -1
Set parameter RelaxLiftCuts to value -1
Set parameter StrongCGCuts to value -1
Set parameter ZeroHalfCuts to value -1
Set parameter CutPasses to value 0
Set parameter Heuristics to value 0
Set parameter Symmetry to value 0


Finally the optimization can be run with the `optimize` method as always.

In [7]:
# Solve the scheduling problem
rail.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Arch Linux")

CPU model: Intel(R) Core(TM) i7-8565U CPU @ 1.80GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Non-default parameters:
TimeLimit  60
Heuristics  0
Symmetry  0
Cuts  0
CutPasses  0
Presolve  1

Optimize a model with 11665 rows, 5700 columns and 30687 nonzeros
Model fingerprint: 0x289043a4
Variable types: 1450 continuous, 4250 integer (4250 binary)
Coefficient statistics:
  Matrix range     [3e-02, 1e+06]
  Objective range  [1e+03, 2e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [9e-02, 1e+06]

User MIP start produced solution with objective 15262.5 (0.03s)
Loaded user MIP start with objective 15262.5

Presolve removed 9186 rows and 3956 columns
Presolve time: 0.45s
Presolved: 2479 rows, 1744 columns, 9105 nonzeros
Found heuristic solution: objective 14969.253214
Variable types: 380 continuous, 1364 integer (1364 binary)

Root relaxatio